In [1]:
import requests
import pandas as pd
import json
from psaw import PushshiftAPI
from nltk import ngrams
from bs4 import *
api = PushshiftAPI()

In [2]:
# # scrape reddit list
# data = []
# def next_page(soup):
#     x = soup.find('ul',{'class':'pager'})
#     s = str(x.select('a')[1])
#     for i in re.findall(r'"([^"]*)"', s):
#         return(i)
# link= "/top-sfw-subreddits"
# i =1
# while i <= 1000:
    
#     url = "https://frontpagemetrics.com" + link
#     page = requests.get(url)

#     soup = BeautifulSoup(page.content, 'html.parser')

#     table = soup.find('table', {'class':"table table-bordered"})
#     table_body = table.find('tbody')

#     rows = table_body.find_all('tr')

#     for row in rows:
#         cols = row.find_all('td')
#         cols = [ele.text.strip() for ele in cols]
#         data.append([ele for ele in cols if ele])
    
#     link = next_page(soup)
#     i+=1

    
    
    
# subreddits = pd.DataFrame(data)
# subreddits.columns=['rank','Subreddits', 'Subscribers']
# subreddits.to_csv("list_of_subreddits.csv")

In [107]:
# function that gets comments dataframe for a subreddit

def get_pushshift_data(data_type, **kwargs):

 
    base_url = f"https://api.pushshift.io/reddit/search/{data_type}/"
    payload = kwargs
    request = requests.get(base_url, params=payload)
    data = request.json().get("data")
    df = pd.DataFrame.from_records(data)[['author','subreddit','body','created_utc']]
    return df

#  function that counts the number of n-grams in a column of a reddit dataframe

def count_grams(df, i):
    df_body = df['body']
    
    grams_dict = {}
    for s in df_body:
        grams = ngrams(s.split(),i)

        for gram in grams:
            gram2 = ' '.join(gram)
            if gram2 in grams_dict:
                grams_dict[gram2] += 1
            else:
                grams_dict[gram2] = 1

    final_df = pd.DataFrame([grams_dict]).T

    return final_df

# collects gram counts from multiple subreddits
def collect_grams(subs,i):
    final_df = pd.DataFrame()
    error_list = []
    for s in subs:
        try:
            df1 = get_pushshift_data(data_type='comment',subreddit=s, after="30d", size = 500)
            count_df = count_grams(df1,i)
            count_df[str(i)+'-gram'] = count_df.index
            count_df['subreddit'] = s
            count_df = count_df.rename(columns={0:'count'})
            final_df = pd.concat([count_df, final_df], ignore_index=True)
        except:
            error_list.append(s)
        
    error = len(error_list)/len(subs)
    print(error)
    
    return final_df, error_list

In [88]:
subs_df = pd.read_csv('list_of_subreddits.csv')
subs_list = [s[3:] for s in subs_df['Subreddits']]

subs_list
subs_100 = subs_list[:100]

In [102]:
collect_grams(subs_100,1)

0.44


(       count      1-gram            subreddit
 56127    266          to                Music
 35102    265         the           philosophy
 56097    260         the                Music
 16664    206         the               bestof
 56214    189         you                Music
 27730    180         the        AdviceAnimals
 9858     173          to  relationship_advice
 52910    172         the            worldnews
 35150    170          to           philosophy
 44412    170         the                books
 33766    160           a           philosophy
 34693    159          of           philosophy
 39086    157          to        Documentaries
 55408    155         and                Music
 33834    151         and           philosophy
 56217    148        your                Music
 16714    143          to               bestof
 55858    142          of                Music
 43269    140           a                books
 48956    139         the               movies
 39041    139

In [108]:
collect_grams(['announcements'],1)

1.0


(Empty DataFrame
 Columns: []
 Index: [], ['announcements'])

In [35]:
x